In [11]:
import pandas as pd
from multiprocessing import Pool
import jellyfish
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
print(dir(jellyfish))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_jellyfish', '_rustyfish', 'damerau_levenshtein_distance', 'hamming_distance', 'jaro_similarity', 'jaro_winkler_similarity', 'levenshtein_distance', 'match_rating_codex', 'match_rating_comparison', 'metaphone', 'nysiis', 'soundex', 'warnings']


In [13]:
left_df = pd.read_csv("/Users/jakepapalia/Downloads/group-project-spring-2024 (1)/Data/GroupProject/entity-resolution/left_dataset.csv")
right_df = pd.read_csv("/Users/jakepapalia/Downloads/group-project-spring-2024 (1)/Data/GroupProject/entity-resolution/right_dataset.csv")

In [21]:
# Preprocessing
left_df = left_df.fillna('')
right_df = right_df.fillna('')

left_df['name'] = left_df['name'].str.lower().str.replace('[^\w\s]', '').str.strip()
left_df['address'] = left_df['address'].str.lower().str.replace('[^\w\s]', '').str.strip()
left_df['city'] = left_df['city'].str.lower().str.replace('[^\w\s]', '').str.strip()
left_df['state'] = left_df['state'].str.lower().str.replace('[^\w\s]', '').str.strip()

right_df['name'] = right_df['name'].str.lower().str.replace('[^\w\s]', '').str.strip()
right_df['address'] = right_df['address'].str.lower().str.replace('[^\w\s]', '').str.strip()
right_df['city'] = right_df['city'].str.lower().str.replace('[^\w\s]', '').str.strip()
right_df['state'] = right_df['state'].str.lower().str.replace('[^\w\s]', '').str.strip()

subset_left = left_df.head(1000)
subset_right = right_df.head(1000)

In [27]:
# function for matching chunks
def find_matches(left_chunk, right_chunk):
    matching_businesses = []
    for _, left_row in left_chunk.iterrows():
        for _, right_row in right_chunk.iterrows():
            # Calculate vectors
            vectorizer = TfidfVectorizer()
            left_vector = vectorizer.fit_transform([left_row['name'], left_row['address'], left_row['city'], left_row['state']])
            right_vector = vectorizer.transform([right_row['name'], right_row['address'], right_row['city'], right_row['state']])
            # Calculate cosine similarity
            similarity_score = cosine_similarity(left_vector, right_vector)[0][0]
            if similarity_score > 0.8:
                matching_businesses.append((left_row['entity_id'], right_row['business_id'], similarity_score))
    return matching_businesses

In [ ]:
chunk_size = 1000

# Process in chunks
matching_businesses = []
for left_start in range(0, len(left_df), chunk_size):
    left_chunk = left_df.iloc[left_start:left_start + chunk_size]
    for right_start in range(0, len(right_df), chunk_size):
        right_chunk = right_df.iloc[right_start:right_start + chunk_size]
        # Find matches for the current chunks
        matches = find_matches(left_chunk, right_chunk)
        matching_businesses.extend(matches)

matches_df = pd.DataFrame(matching_businesses, columns=['entity_id_left', 'business_id_right', 'similarity_score'])

# number of matches
print(len(matches_df))